In [ ]:
#google colab link:-
#https://colab.research.google.com/drive/1RP1tFr_FQJIPz5qzUljN-i6IsGo2-yQD#scrollTo=wlErfRoKPUOU

In [ ]:
# Cell 1: Mount Google Drive
from google.colab import drive
import os

print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted.")

# Define your base project directory within Google Drive
# IMPORTANT: Replace 'my_finetune_project' with your actual project folder name.
# It's recommended to create this folder in your Drive first (e.g., in "My Drive/Colab Notebooks/my_finetune_project")
BASE_PROJECT_DIR = "/content/drive/My Drive/Colab Notebooks/finetuned_+_rag_project"

# Create the project directory if it doesn't exist
os.makedirs(BASE_PROJECT_DIR, exist_ok=True)
print(f"Base project directory set to: {BASE_PROJECT_DIR}")

# Optional: Change current working directory to your project folder
# This makes it easier to use relative paths later, but absolute paths are safer.
# %cd {BASE_PROJECT_DIR}
# print(f"Changed current working directory to: {os.getcwd()}")

In [ ]:
!pip install faiss-cpu
import os
import shutil
import faiss # Make sure faiss is imported here too for consistency with path definition

global GDRIVE_KNOWLEDGE_BASE_PATH, GDRIVE_FINETUNED_MODEL_PATH, GDRIVE_FAISS_INDEX_PATH, KNOWLEDGE_BASE_PATH_TO_USE

GDRIVE_KNOWLEDGE_BASE_PATH = os.path.join(BASE_PROJECT_DIR, "rephrased_output.json")
GDRIVE_FINETUNED_MODEL_PATH = os.path.join(BASE_PROJECT_DIR, "finetuned_qwen")
GDRIVE_FAISS_INDEX_PATH = os.path.join(BASE_PROJECT_DIR, "my_faiss_index.bin")

LOCAL_TEMP_DIR = "/content/temp_rag_data"
os.makedirs(LOCAL_TEMP_DIR, exist_ok=True)

LOCAL_KNOWLEDGE_BASE_PATH = os.path.join(LOCAL_TEMP_DIR, "rephrased_output.json")

if not os.path.exists(LOCAL_KNOWLEDGE_BASE_PATH) and os.path.exists(GDRIVE_KNOWLEDGE_BASE_PATH):
    print(f"Copying knowledge base from Google Drive to local: {GDRIVE_KNOWLEDGE_BASE_PATH} -> {LOCAL_KNOWLEDGE_BASE_PATH}")
    shutil.copy(GDRIVE_KNOWLEDGE_BASE_PATH, LOCAL_KNOWLEDGE_BASE_PATH)
    print("Knowledge base copied to local storage.")
elif not os.path.exists(GDRIVE_KNOWLEDGE_BASE_PATH):
    print(f"Error: Knowledge base file not found in Google Drive at {GDRIVE_KNOWLEDGE_BASE_PATH}. Please ensure it's uploaded.")
else:
    print(f"Knowledge base already exists locally at {LOCAL_KNOWLEDGE_BASE_PATH}. Skipping copy.")

KNOWLEDGE_BASE_PATH_TO_USE = LOCAL_KNOWLEDGE_BASE_PATH

print(f"Knowledge base path for use: {KNOWLEDGE_BASE_PATH_TO_USE}")
print(f"Fine-tuned model path: {GDRIVE_FINETUNED_MODEL_PATH}")
print(f"FAISS index path (Drive): {GDRIVE_FAISS_INDEX_PATH}")

if not os.path.exists(GDRIVE_FINETUNED_MODEL_PATH):
    print(f"Warning: Fine-tuned model directory not found at {GDRIVE_FINETUNED_MODEL_PATH}.")
    print("Please ensure your fine-tuning script has completed successfully and saved the model to this location.")

In [ ]:
print("Starting RAG & Gradio library installation process...")
!pip install sentence-transformers -q
print("Installed sentence-transformers.")
import torch # Important for checking GPU availability
if torch.cuda.is_available():
    print("GPU detected. Installing faiss-gpu...")
    !pip install faiss-gpu -q
else:
    print("No GPU detected. Installing faiss-cpu...")
    !pip install faiss-cpu -q
print("Installed FAISS.")
!pip install gradio -q
print("Installed Gradio.")
!pip install -U bitsandbytes accelerate -q # Crucial for 8-bit quantization
print("Installed/Upgraded bitsandbytes and accelerate.")
!pip install numpy huggingface_hub -q
print("Installed numpy and huggingface_hub.")
print("\n--- ALL REQUIRED LIBRARY INSTALLATIONS COMPLETE ---")
print("Important: Please RESTART RUNTIME (Runtime -> Restart runtime) now,")
print("then re-run cells from the top (starting with Drive Mount)!")

In [ ]:
# Cell 4: Question Generation Pipeline Functions
import json
import numpy as np
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from peft import PeftModel
import torch
from huggingface_hub import login # Keep if still using for model access
import os

# Load the knowledge base with nested "Introduction" structure
def load_knowledge_base(knowledge_base_path):
    print(f"Attempting to load knowledge base from: {knowledge_base_path}")
    try:
        with open(knowledge_base_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                f.seek(0)
                data = [json.loads(line.strip()) for line in f if line.strip()]
        chunks = []
        if isinstance(data, dict):
            data = [data]
        for item in data:
            for topic_name, sections in item.items():
                for section_name, levels in sections.items():
                    for i in range(1, 4):
                        level_key = f"chunks_level{i}"
                        if level_key in levels and isinstance(levels[level_key], list):
                            for chunk in levels[level_key]:
                                if isinstance(chunk, dict) and "text" in chunk:
                                    chunks.append({
                                        "text": chunk["text"],
                                        "title": topic_name,
                                        "section": section_name,
                                        "level": i
                                    })
                                else:
                                    print(f"Warning: Skipping malformed chunk in {level_key}: {chunk}")
                        elif level_key in levels:
                            print(f"Warning: Expected list for {level_key}, found {type(levels[level_key])}. Skipping.")
        if not chunks:
            raise ValueError("No valid chunks found in knowledge base after parsing.")
        print(f"Loaded {len(chunks)} chunks from knowledge base.")
        return chunks
    except FileNotFoundError:
        raise FileNotFoundError(f"Knowledge base not found at {knowledge_base_path}. Please check the path and ensure it's in your Google Drive or copied locally.")
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON/JSONL format in {knowledge_base_path}: {e}. Inspect the file for malformed lines.")
    except Exception as e:
        raise RuntimeError(f"Error loading knowledge base: {e}")

# NEW: Retrieve chunks based on Topic, Subtopic, and Difficulty Level
def retrieve_chunks_by_criteria(all_chunks, topic, subtopic, level, num_chunks=3):
    # print(f"Retrieving chunks for Topic: '{topic}', Subtopic: '{subtopic}', Level: '{level}'") # Muted for cleaner console

    filtered_chunks = []
    for chunk in all_chunks:
        topic_match = chunk['title'].lower() == topic.lower()
        subtopic_match = chunk['section'].lower() == subtopic.lower()
        level_match = chunk['level'] == level

        if topic_match and subtopic_match and level_match:
            filtered_chunks.append(chunk)

    if not filtered_chunks:
        # print(f"Warning: No chunks found for Topic: '{topic}', Subtopic: '{subtopic}', Level: '{level}'. Returning empty list.") # Muted for cleaner console
        return []

    selected_chunks = random.sample(filtered_chunks, min(num_chunks, len(filtered_chunks)))
    # print(f"Retrieved {len(selected_chunks)} chunks based on criteria.") # Muted for cleaner console
    return selected_chunks

# Load fine-tuned Qwen model (max_new_tokens updated)
# (Keeping this function as is)
def load_qwen_local(finetuned_model_path):
    print(f"Loading fine-tuned Qwen model from: {finetuned_model_path}")
    try:
        base_model_name = "Qwen/Qwen2.5-1.5B-Instruct"

        tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path, trust_remote_code=True)

        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_compute_dtype=torch.float16,
        )
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            quantization_config=bnb_config,
            torch_dtype=torch.float16,
            device_map="auto",
            low_cpu_mem_usage=True,
            trust_remote_code=True
        )
        print("Base Qwen model loaded.")

        model = PeftModel.from_pretrained(base_model, finetuned_model_path, trust_remote_code=True)
        model.eval()

        generator = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=250, # Increased max_new_tokens slightly for potentially longer answers
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            torch_dtype=torch.float16,
        )
        print("Text generation pipeline created.")
        return generator
    except Exception as e:
        raise RuntimeError(f"Failed to load fine-tuned Qwen model: {e}")

# NEW: Generate different types of questions with refined prompts and parsing
def generate_question(topic, subtopic, level, question_type, retrieved_chunks, generator):
    context = "\n".join([f"From {chunk['title']} - {chunk['section']} (Level {chunk['level']}):\n{chunk['text']}" for chunk in retrieved_chunks])

    if not context:
        return "Not enough relevant context to generate a question.", "", ""

    question_output = ""
    blank_val = ""
    correct_ans = ""

    # --- Prompt Engineering for Structured Output ---
    # Using explicit start/end markers and very clear instructions
    # Increased max_new_tokens in generator call for more flexibility

    if question_type == "General Question":
        instruction = "Generate a concise and clear general question based on the provided context. The question should immediately follow the 'QUESTION:' tag."

        prompt_template = f"""You are an AI tutor helping a student learn machine learning.

Context from the textbook:
{context}

Instruction: {instruction}

QUESTION: """ # Added a space after the colon to encourage content generation

    elif question_type == "Fill in the Blanks":
        instruction = "Generate a fill-in-the-blank question from the provided context. Choose a key term or concept to be the blank. Provide the blanked sentence. Then, on a new line, provide ONLY the correct answer for the blank. The answer must be a single word. Ensure the question immediately follows 'FIB_QUESTION:' and the answer immediately follows 'FIB_ANSWER:'"

        prompt_template = f"""You are an AI tutor helping a student learn machine learning.

Context from the textbook:
{context}

Instruction: {instruction}

FIB_QUESTION: """ # Added a space after the colon
        # The prompt also needs to guide the model to put the answer immediately after FIB_ANSWER:
        # This will be handled in how we strip the generated text
        # prompt_template += "\nFIB_ANSWER: " # This line is handled by the model completing the pattern

    elif question_type == "Multiple Choice Question":
        instruction = "Generate a multiple-choice question (MCQ) from the provided context. The question should have one correct answer and three plausible incorrect options. Clearly label the options (A, B, C, D). Then, on a new line, provide ONLY the correct option letter (e.g., 'A', 'B', 'C', or 'D'). Ensure the question immediately follows 'MCQ_QUESTION:' and the answer immediately follows 'MCQ_ANSWER:'"

        prompt_template = f"""You are an AI tutor helping a student learn machine learning.

Context from the textbook:
{context}

Instruction: {instruction}

MCQ_QUESTION: """ # Added a space after the colon
        # prompt_template += "\nA) \nB) \nC) \nD) \nMCQ_ANSWER: " # Model should generate these

    else:
        return "Invalid question type selected.", "", ""

    # Generate response
    response = generator(prompt_template, max_new_tokens=400, do_sample=True, temperature=0.7, top_p=0.9)
    generated_text = response[0]["generated_text"]

    # --- Robust Parsing Logic ---
    # Look for the specific markers to extract the exact parts

    if question_type == "General Question":
        # Expected: "QUESTION: [Your generated question text]"
        if "QUESTION:" in generated_text:
            # We want everything *after* "QUESTION: " until the next instruction or end of text
            start_index = generated_text.find("QUESTION:") + len("QUESTION:")
            question_output = generated_text[start_index:].strip()
            # Clean up any trailing instructions or conversational filler generated by the model
            question_output = question_output.split("Instruction:")[0].split("Context from the textbook:")[0].strip()
            question_output = question_output.split("Question:")[0].strip() # Catch variations
        else:
            question_output = "N/A (Failed to parse General Question)"

    elif question_type == "Fill in the Blanks":
        # Expected: "FIB_QUESTION: [Blanked sentence]\nFIB_ANSWER: [Single-word answer]"
        fib_q_marker = "FIB_QUESTION:"
        fib_a_marker = "FIB_ANSWER:"

        if fib_q_marker in generated_text and fib_a_marker in generated_text:
            q_start = generated_text.find(fib_q_marker) + len(fib_q_marker)
            a_start = generated_text.find(fib_a_marker)

            question_output = generated_text[q_start:a_start].strip() # Get text between Q and A markers
            correct_ans = generated_text[a_start + len(fib_a_marker):].strip().split('\n')[0].strip() # Get only the first line/word after FIB_ANSWER:

            # Attempt to extract blank_val if '_____' is present and answer is single word
            if '_____' in question_output:
                blank_val = correct_ans # The single word answer is the blank value
            else:
                blank_val = "N/A (No blank found)"
        else:
            question_output = "N/A (Failed to parse FIB Question)"
            correct_ans = "N/A (Failed to parse FIB Answer)"
            blank_val = "N/A"

    elif question_type == "Multiple Choice Question":
        # Expected: "MCQ_QUESTION: [Question]\nA) [Option A]\n...\nMCQ_ANSWER: [Letter]"
        mcq_q_marker = "MCQ_QUESTION:"
        mcq_a_marker = "MCQ_ANSWER:"

        if mcq_q_marker in generated_text and mcq_a_marker in generated_text:
            q_start = generated_text.find(mcq_q_marker) + len(mcq_q_marker)
            a_start = generated_text.find(mcq_a_marker)

            question_output = generated_text[q_start:a_start].strip() # Get text between Q and A markers
            correct_ans = generated_text[a_start + len(mcq_a_marker):].strip().split('\n')[0].strip() # Get only the first line/letter after MCQ_ANSWER:
        else:
            question_output = "N/A (Failed to parse MCQ Question)"
            correct_ans = "N/A (Failed to parse MCQ Answer)"

    return question_output, blank_val, correct_ans

# Global variables for single loading (optimization)
_chunks = None
_generator = None

# Main question generation pipeline function
def question_generation_pipeline(topic, subtopic, level, question_type):
    print(f"\n--- Running question generation pipeline for: '{topic}' - '{subtopic}' - Level {level}, Type: '{question_type}' ---")
    try:
        global _chunks, _generator

        if _generator is None:
            print("Initializing Qwen model and loading knowledge base for the first time...")
            # Use the global variables (defined in Cell 2) to pass as arguments
            # Ensure KNOWLEDGE_BASE_PATH_TO_USE and GDRIVE_FINETUNED_MODEL_PATH are defined globally
            # For demonstration, I'll add placeholder definitions here for local testing if not in Colab
            # In a real Colab setup, these would be defined in an earlier cell.
            if 'KNOWLEDGE_BASE_PATH_TO_USE' not in globals():
                print("WARNING: KNOWLEDGE_BASE_PATH_TO_USE not defined. Using dummy path for testing.")
                KNOWLEDGE_BASE_PATH_TO_USE = "dummy_knowledge_base.json" # Replace with your actual path
            if 'GDRIVE_FINETUNED_MODEL_PATH' not in globals():
                print("WARNING: GDRIVE_FINETUNED_MODEL_PATH not defined. Using dummy path for testing.")
                GDRIVE_FINETUNED_MODEL_PATH = "dummy_model_path" # Replace with your actual path

            _chunks = load_knowledge_base(KNOWLEDGE_BASE_PATH_TO_USE)
            _generator = load_qwen_local(GDRIVE_FINETUNED_MODEL_PATH)
            print("Qwen model and knowledge base initialized.")

        # Retrieve chunks based on user criteria
        retrieved_chunks = retrieve_chunks_by_criteria(_chunks, topic, subtopic, level, num_chunks=3)

        if not retrieved_chunks:
            print(f"Warning: No relevant chunks found for Topic: '{topic}', Subtopic: '{subtopic}', Level: '{level}'. Cannot generate question.")
            return "No relevant context found to generate a question for your selection.", "", ""

        # Generate question using the new function
        question_output, blank_val, correct_ans = generate_question(topic, subtopic, level, question_type, retrieved_chunks, _generator)

        # --- Console Output Control ---
        print("\n🔍 Retrieved Chunks:")
        for i, chunk in enumerate(retrieved_chunks, 1):
            print(f"**{i}.** From *{chunk['title']} - {chunk['section']}* (Level {chunk['level']}):")
            print(f"{chunk['text'][:300]}...\n") # Display first 300 chars of retrieved chunk text

        print(f"\n📌 Generated {question_type}:")
        print(f"Question: {question_output}")
        if blank_val and blank_val != "N/A": # Only print if meaningful
            print(f"Blank Value: {blank_val}")
        if correct_ans and correct_ans != "N/A (Failed to parse FIB Answer)": # Only print if meaningful
            print(f"Correct Answer/Option: {correct_ans}")


        return question_output, blank_val, correct_ans

    except Exception as e:
        print(f"Error in question generation pipeline: {e}")
        import traceback
        traceback.print_exc() # Print full traceback for debugging
        return f"Error: {e}", "", "" # Return error to Gradio

# Example usage (for testing in a non-Gradio context)
if __name__ == "__main__":
    # Define dummy paths for local testing IF you don't have Cell 2 values set
    # In a real Colab notebook, ensure these are set in an earlier cell.
    global KNOWLEDGE_BASE_PATH_TO_USE, GDRIVE_FINETUNED_MODEL_PATH
    KNOWLEDGE_BASE_PATH_TO_USE = "dummy_knowledge_base.json" # Replace with your actual path
    GDRIVE_FINETUNED_MODEL_PATH = "dummy_model_path" # Replace with your actual path

    # Create a dummy knowledge base file for testing `load_knowledge_base`
    # This is crucial for local testing if the file doesn't exist
    dummy_kb_content = {
        "Introduction": {
            "Linear Regression": {
                "chunks_level1": [{"text": "Linear regression is a basic type of machine learning.", "id": "lr_intro_e1"}],
                "chunks_level2": [{"text": "Linear regression models the relationship between a dependent variable and one or more independent variables using a linear equation.", "id": "lr_intro_m1"}],
                "chunks_level3": [{"text": "Understanding the assumptions of linear regression, such as linearity, independence, homoscedasticity, and normality of residuals, is crucial for its valid application.", "id": "lr_intro_h1"}]
            },
            "Decision Trees": {
                "chunks_level1": [{"text": "Decision trees are flow-chart like structures.", "id": "dt_intro_e1"}],
                "chunks_level2": [{"text": "A decision tree is a non-parametric supervised learning method used for classification and regression.", "id": "dt_intro_m1"}],
                "chunks_level3": [{"text": "The core algorithm for building decision trees is often based on concepts like Gini impurity or information gain for splitting nodes.", "id": "dt_intro_h1"}]
            }
        },
        "Logistic Regression": {
            "Introduction": {
                "chunks_level1": [{"text": "Logistic regression is used for binary classification problems. It predicts a probability.", "id": "logreg_intro_e1"}],
                "chunks_level2": [{"text": "Probabilistic classification algorithms like Logistic Regression use statistics to determine the most likely category for an input, providing a probability rather than just a direct classification.", "id": "logreg_intro_m1"}],
                "chunks_level3": [{"text": "The logistic function (sigmoid) transforms arbitrary real-valued input into a value between 0 and 1, representing a probability. The logit function is its inverse.", "id": "logreg_intro_h1"}]
            }
        }
    }
    with open(KNOWLEDGE_BASE_PATH_TO_USE, "w") as f:
        json.dump(dummy_kb_content, f)
    print(f"Created dummy knowledge base at {KNOWLEDGE_BASE_PATH_TO_USE}")

    # You would typically load your actual model here.
    # For a local test without a real model, this will fail unless you mock `AutoModelForCausalLM` and `AutoTokenizer`.
    # To run this `if __name__ == "__main__":` block fully without a GPU and real model,
    # you'd need to mock the `transformers` and `peft` parts.
    # For now, let's assume `load_qwen_local` will attempt to load, and if it fails,
    # the error handling will catch it.

    test_cases = [

        {"topic": "Logistic Regression", "subtopic": "Introduction", "level": 2, "type": "Fill in the Blanks"},

    ]

    for case in test_cases:
        # Note: If you don't have the Qwen model locally, these calls will fail unless mocked.
        # This is for demonstrating prompt/parsing logic changes, not full execution without model.
        question_output, blank_val, correct_ans = question_generation_pipeline(
            case["topic"], case["subtopic"], case["level"], case["type"]
        )
        print("\n" + "=" * 80 + "\n") # Larger separator for clarity

In [ ]:
# Cell 5: Gradio Interface
import gradio as gr
import json
import os

# Assume KNOWLEDGE_BASE_PATH_TO_USE is defined in Cell 2
# And question_generation_pipeline, load_knowledge_base, etc. are defined in Cell 4

# --- Helper to dynamically get topics, subtopics, and levels from the knowledge base ---
def get_knowledge_base_structure(knowledge_base_path):
    topics = set()
    subtopics_by_topic = {}
    levels = set()

    try:
        with open(knowledge_base_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        if isinstance(data, dict):
            data = [data]

        for item in data:
            for topic_name, sections in item.items():
                topics.add(topic_name)
                if topic_name not in subtopics_by_topic:
                    subtopics_by_topic[topic_name] = set()
                for section_name, levels_data in sections.items():
                    subtopics_by_topic[topic_name].add(section_name)
                    for i in range(1, 4): # Assuming levels 1, 2, 3 based on your JSON structure
                        level_key = f"chunks_level{i}"
                        if level_key in levels_data and isinstance(levels_data[level_key], list) and levels_data[level_key]:
                             levels.add(i)

    except FileNotFoundError:
        print(f"Knowledge base not found at {knowledge_base_path}. Cannot populate dropdowns.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from {knowledge_base_path}: {e}. Cannot populate dropdowns.")
    except Exception as e:
        print(f"An unexpected error occurred while parsing knowledge base: {e}")

    sorted_topics = sorted(list(topics))
    sorted_subtopics_by_topic = {
        topic: sorted(list(s_topics)) for topic, s_topics in subtopics_by_topic.items()
    }
    sorted_levels = sorted(list(levels))

    return sorted_topics, sorted_subtopics_by_topic, sorted_levels

# Get initial structure
TOPICS, SUBTOPICS_BY_TOPIC, LEVELS = get_knowledge_base_structure(KNOWLEDGE_BASE_PATH_TO_USE)

# --- Gradio Interface Logic ---

def update_subtopics(selected_topic):
    if selected_topic in SUBTOPICS_BY_TOPIC:
        # Return a Gradio update object to change choices and potentially value
        return gr.Dropdown(choices=SUBTOPICS_BY_TOPIC[selected_topic], value=SUBTOPICS_BY_TOPIC[selected_topic][0] if SUBTOPICS_BY_TOPIC[selected_topic] else None, interactive=True)
    else:
        return gr.Dropdown(choices=[], value=None, interactive=True)

def gradio_generate_question(topic, subtopic, level, question_type, progress=gr.Progress()):
    progress(0, desc="Starting generation...")

    # Validate inputs
    if not topic or not subtopic or not level or not question_type:
        progress(1, desc="Error: Missing selections.")
        return gr.Markdown("### Error: Please select a Topic, Subtopic, Difficulty Level, and Question Type."), "", ""

    try:
        level_int = int(level)
    except ValueError:
        progress(1, desc="Error: Invalid difficulty level.")
        return gr.Markdown("### Error: Invalid difficulty level selected. Please choose a valid number."), "", ""

    progress(0.1, desc="Loading Qwen model and knowledge base (first time only)...")
    # Call your question generation pipeline
    # The pipeline internally handles _generator and _chunks caching
    question_output, blank_val, correct_ans = question_generation_pipeline(topic, subtopic, level_int, question_type)

    progress(0.8, desc="Formatting output...")

    # Prepare markdown output for the main display
    # Only include question_output here
    main_display_content = f"### Generated {question_type}\n"
    if question_output:
        main_display_content += f"**Question:** {question_output}\n\n"
    else:
        main_display_content += "**Error or No Question Generated.** Please check your selections and try again.\n\n"

    # These will go into the dedicated Textbox outputs
    # If the question type is General Question, these will be "N/A"
    blank_val_output = blank_val if blank_val else "N/A"
    correct_ans_output = correct_ans if correct_ans else "N/A" # This will contain the direct answer for FIB/MCQ option

    progress(1, desc="Generation complete!")

    # We now return 3 outputs: main_display_content, blank_val_output, correct_ans_output
    return gr.Markdown(main_display_content), gr.Textbox(value=blank_val_output), gr.Textbox(value=correct_ans_output)

# --- Gradio Interface Definition ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# AI Tutor: Question Generation")
    gr.Markdown("Select a topic, subtopic, difficulty, and question type to generate educational content.")

    with gr.Row():
        with gr.Column():
            topic_dropdown = gr.Dropdown(
                label="Select Topic",
                choices=TOPICS,
                value=TOPICS[0] if TOPICS else None,
                interactive=True,
                scale=1
            )
            subtopic_dropdown = gr.Dropdown(
                label="Select Subtopic",
                choices=SUBTOPICS_BY_TOPIC.get(TOPICS[0], []) if TOPICS else [],
                value=SUBTOPICS_BY_TOPIC.get(TOPICS[0], [])[0] if TOPICS and SUBTOPICS_BY_TOPIC.get(TOPICS[0]) else None,
                interactive=True,
                scale=1
            )
        with gr.Column():
            level_dropdown = gr.Dropdown(
                label="Select Difficulty Level",
                choices=LEVELS,
                value=LEVELS[0] if LEVELS else None,
                interactive=True,
                scale=1
            )
            question_type_radio = gr.Radio(
                label="Select Question Type",
                choices=["General Question", "Fill in the Blanks", "Multiple Choice Question"],
                value="General Question",
                interactive=True,
                scale=1
            )
            generate_button = gr.Button("Generate Question", variant="primary", scale=2)

    # Output area - Simplified
    with gr.Column():
        # Main question display (includes question and options for MCQ)
        output_question = gr.Markdown(label="Generated Question")

        # Display for the correct answer (will be N/A for General Questions)
        output_answer = gr.Textbox(label="Correct Answer / Blank Value", interactive=False)
        # We can combine blank_val_output and correct_ans_output into one text box if desired.
        # If we separate them, it might be more explicit. For simplicity, I'm combining them.
        # If blank_val_output is used, correct_ans_output will be blank, and vice-versa, or N/A.

    # Define interactions
    topic_dropdown.change(
        fn=update_subtopics,
        inputs=topic_dropdown,
        outputs=subtopic_dropdown
    )

    generate_button.click(
        fn=gradio_generate_question,
        inputs=[topic_dropdown, subtopic_dropdown, level_dropdown, question_type_radio],
        outputs=[output_question, output_answer] # Only pass two outputs now
    )

# Launch the Gradio interface
print("Launching Gradio interface...")
demo.launch(debug=True, share=True)